# Attention

注意力机制

找到哪更应该被注意 - 找注意力

然后将原来的内容，按照注意力加权求和，得到真正要被注意的内容

- 今天天气不错
- 今天太糟糕了
- 今天天气
- 不错

sequence to sequence

序列到序列

我 爱 你

I love you

encoder(我爱你) -> 对上文的理解

- t0: decoder(对上文的理解, 起始符号) -> I
- t1: decoder(对上文的理解', I) -> love
- t2: decoder(对上文的理解'', love) -> you
- t3: decoder(对上文的理解''', you) -> 终止字符


对上文的理解 -> 对每个字的理解 -> 【我的理解， 爱的理解， 你的理解】

找到注意力(【我的理解， 爱的理解， 你的理解】: Key, 起始字符: Query） -> 注意力 = [0.98, 0.01, 0.01]

注意力 * 每个字的理解 = sum( [0.98, 0.01, 0.01] * 【我的理解， 爱的理解， 你的理解】: Value )
                   = sum( 0.98 * 我 + 0.01 * 爱 + 0.01 * 你 )
                   = 根据起始字符进行注意力后的理解

根据起始字符进行注意力后的理解 = 注意理解(【我的理解， 爱的理解， 你的理解】, 起始字符)

- t0: decoder(根据起始字符进行注意力后的理解) -> I
- t1: decoder(根据I进行注意力后的理解, I) -> love
- t2: decoder(根据love进行注意力后的理解, love) -> you
- t3: decoder(根据you进行注意力后的理解, you) -> 终止字符


【我的理解， 爱的理解， 你的理解】


```
        我的理解， 爱的理解， 你的理解
我的理解      1.0    0.0      0.0
爱的理解      0.9    0.01      0.0
你的理解      0.0    0.0       0.0
```

【我的理解'， 爱的理解'， 你的理解'】



In [1]:
import tensorflow as tf

In [2]:
batch_size = 4
sequence_length = 10
vector_size = 32

x = tf.random.uniform((batc
                       h_size, sequence_length, vector_size))

In [3]:
x.shape

TensorShape([4, 10, 32])

In [16]:
def self_attention(x):
    key = tf.keras.layers.Dense(vector_size)(x)
    value = tf.keras.layers.Dense(vector_size)(x)
    query = tf.keras.layers.Dense(vector_size)(x)

    atten = tf.matmul(query, key, transpose_b=True)
    atten = atten / tf.sqrt(tf.cast(vector_size, tf.float32))
    atten = tf.nn.softmax(atten)
    y = tf.matmul(atten, value)
    return y

In [17]:
self_attention(x).shape

TensorShape([4, 10, 32])

In [18]:
batch_size = 4
sequence_length = 10
vector_size = 32
heads = 4

x = tf.random.uniform((batch_size, sequence_length, vector_size))

In [41]:
def multi_head_self_attention(x):
    x_mh = tf.reshape(x, [batch_size, sequence_length, heads, vector_size // heads])
    x_mh = tf.transpose(x_mh, (0, 2, 1, 3))
    key = tf.keras.layers.Dense(vector_size // heads)(x_mh)
    value = tf.keras.layers.Dense(vector_size // heads)(x_mh)
    query = tf.keras.layers.Dense(vector_size // heads)(x_mh)
    atten = tf.matmul(query, key, transpose_b=True)
    atten = atten / tf.sqrt(tf.cast(vector_size, tf.float32))
    atten = tf.nn.softmax(atten)
    y = tf.matmul(atten, value)
    y = tf.transpose(y, [0, 2, 1, 3])
    y = tf.reshape(y, (batch_size, sequence_length, vector_size))
    return y

In [42]:
multi_head_self_attention(x).shape

TensorShape([4, 10, 32])

In [43]:
class MultiHeadSelfAttention(tf.keras.Model):
    def __init__(self, vector_size, heads=1):
        super(MultiHeadSelfAttention, self).__init__()
        self.vector_size = vector_size
        self.heads = heads
        self.key = tf.keras.layers.Dense(vector_size // heads)
        self.value = tf.keras.layers.Dense(vector_size // heads)
        self.query = tf.keras.layers.Dense(vector_size // heads)

    def call(self, x):
        batch_size = x.shape[0]
        sequence_length = x.shape[1]
        heads = self.heads
        vector_size = self.vector_size

        x_mh = tf.reshape(x, [batch_size, sequence_length, heads, vector_size // heads])
        x_mh = tf.transpose(x_mh, (0, 2, 1, 3))
        key = tf.keras.layers.Dense(vector_size // heads)(x_mh)
        value = tf.keras.layers.Dense(vector_size // heads)(x_mh)
        query = tf.keras.layers.Dense(vector_size // heads)(x_mh)
        atten = tf.matmul(query, key, transpose_b=True)
        atten = atten / tf.sqrt(tf.cast(vector_size, tf.float32))
        atten = tf.nn.softmax(atten)
        y = tf.matmul(atten, value)
        y = tf.transpose(y, [0, 2, 1, 3])
        y = tf.reshape(y, (batch_size, sequence_length, vector_size))
        return y

In [44]:
batch_size = 4
sequence_length = 10
vector_size = 32
heads = 4

x = tf.random.uniform((batch_size, sequence_length, vector_size))

In [45]:
attention_model = MultiHeadSelfAttention(vector_size, heads)

In [47]:
attention_model(x).shape

TensorShape([4, 10, 32])

In [48]:
x.shape

TensorShape([4, 10, 32])